# Statistics on the output data. 

In [248]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import lizard
import subprocess as sub
from pylibsrcml import srcml

df = pd.read_csv('../data/output-tunslip.csv')

if not os.path.exists("figure"):
    os.mkdir("figure")

df.columns

Index(['File', 'Line', 'Column', 'DefaultLevel', 'Level', 'Category', 'Name',
       'Warning', 'Suggestion', 'Note', 'CWEs', 'Context', 'Fingerprint',
       'ToolVersion', 'RuleId', 'HelpUri'],
      dtype='object')

In [249]:
dfp = df[['Category', 'Name', 'CWEs']].groupby(['Category', 'Name', 'CWEs'], group_keys=False).size().reset_index(name='Count')
print(dfp)
fig = px.sunburst(dfp, path=['Category', 'Name', 'CWEs'], values='Count')
# fig.write_image("figure/vul_statistics.pdf")
fig.show()
df.head(3)

    Category       Name              CWEs  Count
0     buffer       char  CWE-119!/CWE-120      6
1     buffer     getopt   CWE-120, CWE-20      1
2     buffer     memcpy           CWE-120     11
3     buffer       read   CWE-120, CWE-20      1
4     buffer     strcat           CWE-120      1
5     buffer     strcpy           CWE-120      4
6     buffer    strncpy           CWE-120      1
7     format  vsnprintf           CWE-134      1
8    integer       atoi           CWE-190      2
9       misc       open           CWE-362      2
10  obsolete     usleep           CWE-676      1
11     shell     system            CWE-78      1


,File,Line,Column,DefaultLevel,Level,Category,Name,Warning,Suggestion,Note,CWEs,Context,Fingerprint,ToolVersion,RuleId,HelpUri
0,data/projects/contiki-2.4/tools/tunslip.c,438,3,4,4,format,vsnprintf,If format strings can be influenced by an atta...,Use a constant for the format specification.,NaN,CWE-134,"vsnprintf(cmd, sizeof(cmd), fmt, ap);",5a6ebc53325719b0f59e24f31e308667e9337289bfbc55...,2.0.19,FF1019,https://cwe.mitre.org/data/definitions/134.html
1,data/projects/contiki-2.4/tools/tunslip.c,442,10,4,4,shell,system,This causes a new program to execute and is di...,try using a library call that implements the s...,NaN,CWE-78,return system(cmd);,f3f1fc4af80877aebfc643d7ec0da9b9e96eccff3cad56...,2.0.19,FF1044,https://cwe.mitre.org/data/definitions/78.html
2,data/projects/contiki-2.4/tools/tunslip.c,753,3,4,4,buffer,strcat,Does not check for buffer overflows when conca...,"Consider using strcat_s, strncat, strlcat, or ...",NaN,CWE-120,"strcat(t, dev);",901ebb10ee39a0103c1035a30e7a1a1f2db2bf8d9bf07e...,2.0.19,FF1005,https://cwe.mitre.org/data/definitions/120.html


## Grepping functions from the vulnerability context of the file.

In [250]:
# Function Under Construction:
def srcML_funs(file):
    """ finds function blocks of the given file
    """ 
    fun_ptn = "string(//src:function)"
    funblk_ptn = "string((//src:function/src:name))"
    # file_ptn = "string(//src:unit/@filename)" 

    # cmd = sub.Popen(["srcml", "--xpath", fun_ptn, file], stderr=sub.STDOUT)
    # out, err = cmd.communicate()
    cmd = ["srcml", "--xpath", funblk_ptn, xml_file]
    process = sub.Popen(cmd, stderr=sub.STDOUT)
    return process

file = '../' + df.File[0]
xml_file = "a.cpp.xml"

file_str = ''
with open(file) as f:
    file_str = f.read()

## Find the metrics of the given file

In [283]:
import itertools

# Without checking at CSV output of flawfinder. 
# def find_metrics(source_file):
#     """ split the given file into a list of function blocks and return their metrics into a dataframe
#     """ 
#     df = pd.DataFrame()
#     with open(source_file, "r") as fp:  
#         liz_file = lizard.analyze_file.analyze_source_code(source_file,  fp.read())
        
#         for x in range(len(liz_file.function_list)):
#             fun_metrics = liz_file.function_list[x].__dict__
#             start=fun_metrics['start_line']
#             end=fun_metrics['end_line']
            
#             fp.seek(0) # move header to the initial point of the file
#             df_fun = pd.DataFrame.from_dict(fun_metrics)
#             lines = [line for line in itertools.islice(fp, start, end)]
#             df_fun['code'] =  fun_metrics['long_name'] + ''.join(lines) 
#             df = pd.concat([df, df_fun])
            
#     # <guru> I think there is a problem in lizard detecting the correct full_parameters 
#     # either we have to concatenate two lines of full_parameters or ignore it and take it from long_name if needed. 
#     # drop['full_parameters', 'fan_in', 'fan_out', 'general_fan_out'] because lizard has not properly implemented these parameters yet.
#     cols_filter = ['full_parameters', 'fan_in', 'fan_out', 'general_fan_out']
#     df = df.drop(cols_filter, axis=1).drop_duplicates().reset_index(drop=True)
#     print('Shape of the dataframe: ', df.shape)
#     return df

# df_met = find_metrics(file)
# df_met.head(5)

Shape of the dataframe:  (20, 10)


,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,top_nesting_level,code
0,3,43,336,relay_dhcp_to_server,"relay_dhcp_to_server( struct ip * ip , int len)",162,224,../data/projects/contiki-2.4/tools/tunslip.c,0,"relay_dhcp_to_server( struct ip * ip , int len..."
1,14,91,725,relay_dhcp_to_client,relay_dhcp_to_client( int slipfd),229,338,../data/projects/contiki-2.4/tools/tunslip.c,0,relay_dhcp_to_client( int slipfd){\n struct d...
2,5,18,118,ip4sum,"ip4sum( u_int16_t sum , const void * _p , u_in...",344,362,../data/projects/contiki-2.4/tools/tunslip.c,0,"ip4sum( u_int16_t sum , const void * _p , u_in..."
3,15,34,256,check_ip,"check_ip( const struct ip * ip , unsigned ip_len)",365,413,../data/projects/contiki-2.4/tools/tunslip.c,0,"check_ip( const struct ip * ip , unsigned ip_l..."
4,8,12,90,is_sensible_string,"is_sensible_string( const unsigned char * s , ...",416,427,../data/projects/contiki-2.4/tools/tunslip.c,0,"is_sensible_string( const unsigned char * s , ..."


# Fetching the functions which have given line context/statement.

In [374]:
import itertools
    
    
def file2metrics(source_file, df_flaw):
    """ split the given file into a list of function blocks and return their metrics into a dataframe
    """ 
    lines = list(set(list(df_flaw.Line)))
    df = pd.DataFrame()
    with open(source_file, "r") as fp:  
        liz_file = lizard.analyze_file.analyze_source_code(source_file,  fp.read())
        
        for x in range(len(liz_file.function_list)):
            
            fun_metrics = liz_file.function_list[x].__dict__
            df_fun = pd.DataFrame()
            df_fun = pd.DataFrame.from_dict(fun_metrics)
            
            start = int(fun_metrics['start_line'])
            end = int(fun_metrics['end_line']) 
            fp.seek(0) # move header to the initial point of the file

            vul_statement, cwe, vul_bool = '', [], False
            
            for l in lines:      
                code_lines = [line for line in itertools.islice(fp, start, end)]
                df_fun['code'] =  fun_metrics['long_name'] + ''.join(code_lines) 
                
                # check if the vulnerability content/statement appear in the function block or not.
                if start <= l <= end:
                    vul_bool =  True 
                    # vul_statement = vul_statement + ' \n ' +  df_flaw[df_flaw.Line==l]['Context'].values[0]
                    vul_statement = df_flaw[df_flaw.Line==l]['Context'].values[0]
                    vul_type = df_flaw[df_flaw.Line==l]['CWEs'].values[0]
                    cwe.append((vul_type, vul_statement))

                df_fun['CWEs'] = str(cwe)
                # df_fun['vul_statements'] = vul_statement
                
            df_fun['is_vul'] = vul_bool  
            df = pd.concat([df, df_fun])
            
    # <guru> I think there is a problem in lizard detecting the correct full_parameters 
    # either we have to concatenate two lines of full_parameters or ignore it and take it from long_name if needed. 
    # drop['full_parameters', 'fan_in', 'fan_out', 'general_fan_out'] because lizard has not properly 
    # implemented these parameters yet.
    
    cols_filter = ['full_parameters', 'fan_in', 'fan_out', 'general_fan_out']
    df = df.drop(cols_filter, axis=1).drop_duplicates().reset_index(drop=True)
    print('Shape of the dataframe: ', df.shape)
    return df


df_met = file2metrics(file, df)
df_met

Shape of the dataframe:  (20, 12)


,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,top_nesting_level,code,CWEs,is_vul
0,3,43,336,relay_dhcp_to_server,"relay_dhcp_to_server( struct ip * ip , int len)",162,224,../data/projects/contiki-2.4/tools/tunslip.c,0,"relay_dhcp_to_server( struct ip * ip , int len)","[('CWE-120', ' memcpy(&m, inm, DHCP_BASE_LEN)...",True
1,14,91,725,relay_dhcp_to_client,relay_dhcp_to_client( int slipfd),229,338,../data/projects/contiki-2.4/tools/tunslip.c,0,relay_dhcp_to_client( int slipfd),"[('CWE-120', ' memcpy(pkt.m.options, inm.opti...",True
2,5,18,118,ip4sum,"ip4sum( u_int16_t sum , const void * _p , u_in...",344,362,../data/projects/contiki-2.4/tools/tunslip.c,0,"ip4sum( u_int16_t sum , const void * _p , u_in...",[],False
3,15,34,256,check_ip,"check_ip( const struct ip * ip , unsigned ip_len)",365,413,../data/projects/contiki-2.4/tools/tunslip.c,0,"check_ip( const struct ip * ip , unsigned ip_len)",[],False
4,8,12,90,is_sensible_string,"is_sensible_string( const unsigned char * s , ...",416,427,../data/projects/contiki-2.4/tools/tunslip.c,0,"is_sensible_string( const unsigned char * s , ...",[],False
5,1,11,64,ssystem,"ssystem( const char * fmt , ...)",433,443,../data/projects/contiki-2.4/tools/tunslip.c,0,"ssystem( const char * fmt , ...)","[('CWE-119!/CWE-120', ' char cmd[128];'), ('C...",True
6,31,100,608,serial_to_tun,"serial_to_tun( FILE * inslip , int outfd)",455,585,../data/projects/contiki-2.4/tools/tunslip.c,0,"serial_to_tun( FILE * inslip , int outfd)","[('CWE-119!/CWE-120', ' unsigned char inbuf...",True
7,2,7,37,slip_send,"slip_send( int fd , unsigned char c)",591,597,../data/projects/contiki-2.4/tools/tunslip.c,0,"slip_send( int fd , unsigned char c)",[],False
8,6,17,91,slip_flushbuf,slip_flushbuf( int fd),606,625,../data/projects/contiki-2.4/tools/tunslip.c,0,slip_flushbuf( int fd),[],False
9,9,42,232,write_to_serial,"write_to_serial( int outfd , void * inbuf , in...",628,681,../data/projects/contiki-2.4/tools/tunslip.c,0,"write_to_serial( int outfd , void * inbuf , in...",[],False
